# **Tutorial GROVER - DNA Language Model**

Melissa Sanabria, Jonas Hirsch, Pierre M. Joubert, Anna R. Poetsch

Biomedical Genomics, Biotechnology Center, Center for Molecular and Cellular Bioengineering, Technische Universität Dresden  
melissa.sanabria@tu-dresden.de arpoetsch@gmail.com


# Overview

This tutorial will show you how to use the DNA language model GROVER to perform fine-tuning tasks to investigate genome biology.

[GROVER](https://www.nature.com/articles/s42256-024-00872-0) ("Genome Rules Obtained Via Extracted Representations") is a foundation DNA language model with an optimized vocabulary for the human genome. It has been pre-trained on the human genome and needs to be trained a second time, or fine-tuned, to perform specific tasks.
   
   
For this tutorial we chose an end-to-end example to fine-tune the pre-tained GROVER to predict DNA binding by the CCCTC-Binding Factor (CTCF). The task is to recognize which sites that contain a CTCF binding motif are actually bound by the protein. We will be using ChIP-seq data from HepG2 cells obtained from ENCODE.  

Throughout this tutorial, you will find ***MODIFY*** signs that indicate pieces of code that are task specific, meaning that they are specific for CTCF binding site prediction, and you can modify them for your desired task.

**WARNING**: Even though Google Colab gives you access to a GPU, which is necessary for this computationally expensive work, the free version only gives you access to a very simple device and the training might take a while.

In the version of Colab, which is free of charge, notebooks can run for a maximum of 12 hours, depending on availability and your usage patterns.  

## Setup

First, save a copy of this notebook in your Google Drive by navigating to 'File' then 'Save a copy in Drive...'.

Once you've opened your own copy, make sure you have enabled the GPU runtime for Google Colab by navigating to the menu 'Runtime', select 'Change runtime type' and set the runtime to 'T4 GPU'.


## Import necessary packages

In [2]:
!pip install wget
!pip install pandas scikit-learn transformers torch
!pip install accelerate

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [3]:
import pandas as pd
import wget
import os
import numpy as np

from sklearn.metrics import matthews_corrcoef, accuracy_score, precision_score, recall_score, f1_score

import transformers
from transformers import AutoTokenizer, Trainer, AutoModelForSequenceClassification, TrainingArguments
from torch.utils.data import Dataset

ModuleNotFoundError: No module named 'pandas'

# Prepare Data

**MODIFY!**
Create Dataset with cancer variant

# Create dataset

From now on, we come back to general instructions that apply for any fine-tuning task.

Please remember that to do this yourself you will need a data frame with at least two columns: label, sequence

In [4]:
import requests

# Télécharger les variants pathogènes de ClinVar
url = "https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz"
response = requests.get(url, stream=True)

with open("clinvar_cancer.vcf.gz", "wb") as f:
    f.write(response.content)

print("Fichier VCF téléchargé avec succès !")

Fichier VCF téléchargé avec succès !


In [5]:
import gzip
import requests
import csv
import random

# 📌 PARAMETERS
VCF_FILE = "clinvar_cancer.vcf.gz"  # Your compressed VCF file
TSV_OUTPUT = "cancer_variants.tsv"
FLANK_SIZE = 500  # Number of nucleotides before/after the mutation
SAMPLE_SIZE = 200  # Number of variants to process (0 for all)

# 📌 1️⃣ READ VARIANTS FROM A COMPRESSED `.vcf.gz` FILE
def read_vcf_gz(vcf_file):
    """Reads a compressed .vcf.gz file and extracts cancer variants with risk factor"""
    variants = []
    with gzip.open(vcf_file, "rt") as f:  # Open in text mode
        for line in f:
            if line.startswith("#"):  # Skip header lines
                continue
            fields = line.strip().split("\t")
            chrom = fields[0]
            pos = int(fields[1])
            ref = fields[3]
            alt = fields[4].split(",")[0]  # Take the first ALT allele
            info = fields[7]  # INFO field

            # Check if the variant has a risk factor in the CLNSIG field
            if "CLNSIG=" in info:
                clnsig = info.split("CLNSIG=")[1].split(";")[0]
                if "risk" in clnsig :
                    variants.append((chrom, pos, ref, alt))
    return variants

# 📌 2️⃣ GET THE REFERENCE SEQUENCE AROUND THE VARIANT
def get_sequence(chrom, pos, flank=500):
    """Fetches the reference DNA sequence ±500 nucleotides around the variant"""
    start = max(1, pos - flank)
    end = pos + flank
    url = f"https://rest.ensembl.org/sequence/region/human/{chrom}:{start}-{end}?content-type=text/plain"
    
    response = requests.get(url)
    if response.status_code == 200:
        return response.text.strip(), start, end
    else:
        print(f"❌ Error fetching sequence for {chrom}:{pos}")
        return None, None, None

# 📌 3️⃣ INSERT MUTATION INTO THE SEQUENCE
def mutate_sequence(sequence, ref, alt, flank=500):
    """Inserts mutation (SNP or Indel) into the reference DNA sequence"""
    center = flank  # Position of the variant in the extracted sequence
    if sequence[center:center + len(ref)] != ref:
        print(f"⚠ Mismatch: Expected {ref} but found {sequence[center:center + len(ref)]}")
        return None
    mutated_sequence = sequence[:center] + alt + sequence[center + len(ref):]
    return mutated_sequence

# 📌 4️⃣ SAVE TO A TSV FILE
def write_tsv(output_file, data):
    """Writes variant data to a TSV file"""
    with open(output_file, "w", newline="") as f:
        writer = csv.writer(f, delimiter="\t")
        writer.writerow(["chromosome", "start_of_bin", "end_of_bin", "label", "sequence"])
        writer.writerows(data)

# 📌 5️⃣ RUN THE FULL PIPELINE
def process_variants(vcf_file, output_tsv, flank_size=500, sample_size=0):
    """Full pipeline to generate mutated sequences in TSV format"""
    variants = read_vcf_gz(vcf_file)
    print(f"🔍 Found {len(variants)} variants with risk factor.")

    # Apply random sampling if requested
    if sample_size > 0:
        sample_size = min(sample_size, len(variants))
        variants = random.sample(variants, sample_size)
        print(f"🔍 Randomly selected {sample_size} variants.")

    output_data = []

    for chrom, pos, ref, alt in variants:
        print(f"📌 Processing variant {chrom}:{pos} {ref}>{alt}")

        # Get reference sequence
        seq, start, end = get_sequence(chrom, pos, flank_size)
        if not seq:
            continue  # Skip this variant if error occurs

        # Insert mutation
        mutated_seq = mutate_sequence(seq, ref, alt, flank_size)
        if not mutated_seq:
            continue  # Skip if mutation failed

        # Add both mutated and reference sequences
        output_data.append([chrom, start, end, 1, mutated_seq])
        output_data.append([chrom, start, end, 0, seq])

    # Write to TSV file
    write_tsv(output_tsv, output_data)
    print(f"✅ Cancer variant sequences with risk factor saved to {output_tsv}")

# 📌 RUN THE SCRIPT
process_variants(VCF_FILE, TSV_OUTPUT, FLANK_SIZE, SAMPLE_SIZE)

🔍 Found 800 variants with risk factor.
🔍 Randomly selected 200 variants.
📌 Processing variant 7:44152381 T>A
📌 Processing variant 12:40340400 G>A
📌 Processing variant X:83874085 G>T
📌 Processing variant 4:6302355 C>T
📌 Processing variant 17:37731730 T>C
📌 Processing variant 4:83267659 A>G
📌 Processing variant 7:44149813 G>A
📌 Processing variant 10:71819104 T>G
📌 Processing variant 5:112839514 T>A
📌 Processing variant 11:17387583 T>C
📌 Processing variant 4:163148340 C>A
📌 Processing variant 3:9944556 G>A
📌 Processing variant 3:44775355 C>T
📌 Processing variant 20:59301614 A>AG
📌 Processing variant 3:30644840 A>G
📌 Processing variant 1:21290752 G>T
📌 Processing variant 12:6018667 T>C
📌 Processing variant 2:232791426 A>G
📌 Processing variant 1:54032127 C>T
📌 Processing variant 6:31575186 G>A
📌 Processing variant 11:2159893 T>A
📌 Processing variant 17:78133937 CT>C
📌 Processing variant 1:161039733 G>A
📌 Processing variant 10:43109146 C>A
📌 Processing variant 9:22125504 G>C
📌 Processing var

⚠ Mismatch: Expected TGGAAGAAACATTGCCAGACGTGGCCATTTGCAAATGTCGTTTCCTGGTCTGTGTCTTTTGATTCTCCTGACAGCATATTTGAGTTAATTTTTTAATGTGTAAGGTATGAGTGGAGGTCCTCATTAAAAATTGTTTAGGCTGGCTGTATGGCTCATGCCTGTAATCACAGCACTTTGAGAGGCTGAAGTGGGAGGATCATTTGAGCCCAGGAGTTTGAGACCGGCTGGGCTAACATGGCGAGACCCCGTCTCTACAAAAAATAAGAAAATTAGCTGGGTGTGGCACCACGCATCTGTAGTCTCAGTTGTACAGGAGGCTGAGGCCAGAGGATCACTTGAGCCCAGGAGATTGAGGCTGCAGTGAGCCATGATTGCAGTAAGATGTGATCACATCCAGTTACATTTGATTTTCAGATAAACAACAGAAGTTTAGTGGAGGCTGGGGGCAGTGGCTCACAGCTGTAATCCCAGCACTTTAGGAAGCTGAGGCAGGTGGATCACCTGAGGTCAGGAGTTCGAGACCAGCCTGGCCAATATGATGAAACCGTGTCTCTACTAAAAATACAAAAAAATTAGCCGGGCGTGGTGGCGGGCGCCTATAATCCCAGCTACTCAGAAGGCTGAGGCAGGAGAATCGCTTGAACCCGGGAGACGGAGGTTGCAGTGAGCCAAGATTGTGCTGCTGCACTCCAGCCTGGGTGATAAGAGTGAAACTCCATCTCAAAAAAAAAAAAAAAAAAAAAACAAATTGGGGTTGTCTGTCTTTTCTCATTGTTTCCATTGTTTCTGGATCTGTGTGTCAGAGATGTGTGCTGCACACATCCTTTCTCCGCCTAGCTTGGTTTTCACTCTTGGTGGTCGCAGACATCTTGGGGGCTACTTGAAGGAACTCTAAGCCGACCAGAGGCTGGGCTGGACTCGCCGTTTGGCGCCCAGCCGGAGACCCCAGGGCCACCCTCCCCGGTTCCATCCTGGGAGA

In [6]:
CTCF_dataset = pd.read_csv(TSV_OUTPUT, sep='\t')
CTCF_dataset

NameError: name 'pd' is not defined

In [7]:
#example sequence
CTCF_dataset['sequence'][0]

'GGAGAGTGTTGCTTGGACCAGGGGCTGGCCATAAGGATGGAGAGAAGGAGGTGATTCTTATAACATCATGGGAGAATGCAGGTGTTGATGGATTGAATGTGAGAAAGGGATTCCCAGGCAGGACCCCTGCACTTGGTTTAATGAAATTCTTAATAATTTCTGAGCGAGGGGCCGTGTCTCTTCATTTTGCGCTGGGCTTCGCAGGTCTCTGGTTTTTGCAATGCTGGAGTGACGCCTCTGGCTCATATATCTGCCTGCCCCCACCCCCCAGCCCAATCTTTGCAAAGGGAATTGGGCAGTGGGCACATGGAGGTGGGGCTCTCTCTCGTGTTCATCCTGCGAGACTCAGGAGCCCCTCTCTTCCATTTTGCCCTAAATCCCAGCCTCTTACAATCATTTCTTGGTGTATCACGGGCGCATCCTCCTGGAAGACCCCGTCGGGCTTCTGCTTCTCCAGGATCAGCCATTTAACAGCCCCGCAGAGGACTTGGGAGTCGATGACGATGAGGTTGACAGCCAGAGAGAAGACCTTGACCACGTAGGCGGTCAGCCTGGAGTGGGCACAGAGCATGAGCCAATCGGCTCTGAGATCCAGAGACTGCCATGTCAACCAGCCAATGAGCGTGGGGGAGGGACCAGGGCCTGGCCCAGTTTGCCTGGTTTGCCTTCCCAGGCCCCAGGACCCAGCTGTTGTATGCGGTCCGTGCTTAAGGATGCTTAATGACCGCCGGCCACTCAGCCAGCGCTTGCCTGGACTCTGCAGGTCCAGGGCTGTTTGGGCAAGGCTGGCCTAAGGGACCACCCCTGGCCAGGGTCCGGGCCCTGCTGGGGGTTGAGGGTGGGGAGTATGCATGGCCTGAGCTGGCTGTTGGGACTCACCAGGTGCTGGGTGCCCGTTTCACGAAGGCCGCAAAGGCAGAGCTGGGTTGTCTGAAGGCCAGCTGCTGGGTGTACCCTGCAGAGAAGAGAGAGGAACCCCCAAAAGAGCCGGGGCTGAGC

After getting our dataset, we need to split the samples into train, validation and test. We will go for the standard partitions, 80% for training, 10% for testing and 10% for validation.

In [8]:
train = CTCF_dataset.sample(frac=0.6, random_state=0)
validation = CTCF_dataset.drop(train.index)
test = validation.sample(frac=0.5, random_state=0)
validation = validation.drop(test.index)

train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)
test = test.reset_index(drop=True)

# GROVER in action

So far, we have not mentioned any Language Model terminology. Now, we need to change our sequence from nucleotides to tokens (or words). For this, we will download the tokenizer available in the higgingface project for GROVER. For this, we can use the model name *PoetschLab/GROVER*

In [9]:
from transformers import PreTrainedTokenizerFast

ImportError: cannot import name 'Tensor' from 'torch' (unknown location)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("PoetschLab/GROVER")

Then, we need to download the pre-trained GROVER model.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("PoetschLab/GROVER")

In order to make your work easier, we create a class to process the Dataset created for Grover.


In [ ]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, texts, labels, tokenizer):

        super(SupervisedDataset, self).__init__()

        sequences = [text for text in texts]

        output = tokenizer(
            sequences,
            add_special_tokens=True,
            max_length=310,
            padding="longest",
            return_tensors="pt",
            truncation=True
        )

        self.input_ids = output["input_ids"]
        self.attention_mask = output["attention_mask"]
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i):
        return dict(
            input_ids=self.input_ids[i],
            labels=self.labels[i],
            attention_mask=self.attention_mask[i]
        )

In [ ]:
train_dataset = SupervisedDataset(train.sequence, train.label, tokenizer)
test_dataset = SupervisedDataset(test.sequence, test.label, tokenizer)
val_dataset = SupervisedDataset(validation.sequence, validation.label, tokenizer)

## Train model

For the training process we choose:

*   an Adam optimizer but you can explore the [optimizers](https://huggingface.co/transformers/v3.0.2/main_classes/optimizer_schedules.html) available in huggingface
*   learning rate of 0.000001 since for fine-tuning tasks it is better to have a very low learning rate
*   a total number of epochs of 4

A higher number of epochs is ideal but Colab limits the amount of time you can run some code. We advice you to try with your own GPU resources with at least 10 epochs.

In order to make your work easier, we create a method to compute five classification metrics. Here we will see some metrics such as accuracy, f1 score, precision and recall. You can explore the [sklearn metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) to choose other options.

In [ ]:
def calculate_metric_with_sklearn(logits: np.ndarray, labels: np.ndarray):
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(
            labels, predictions, average="macro", zero_division=0
        ),
        "matthews_correlation": matthews_corrcoef(
            labels, predictions
        ),
        "precision": precision_score(
            labels, predictions, average="macro", zero_division=0
        ),
        "recall": recall_score(
            labels, predictions, average="macro", zero_division=0
        ),
    }

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):  # Unpack logits if it's a tuple
        logits = logits[0]
    return calculate_metric_with_sklearn(logits, labels)

In [ ]:
train_args = TrainingArguments(seed = 42,
                               output_dir=".",
                               per_device_train_batch_size=8,
                               eval_strategy="epoch",
                               learning_rate=0.000001,
                               num_train_epochs=4
                               )
trainer = transformers.Trainer(
                                model=model,
                                tokenizer=tokenizer,
                                compute_metrics=compute_metrics,
                                train_dataset=train_dataset,
                                eval_dataset=val_dataset,
                                args = train_args
                                )
trainer.train()

## Test model

After training the model, we can see the performance of our model on the test set, which are samples that the model has not previously seen.

In [ ]:
results = trainer.evaluate(eval_dataset=test_dataset)

In [ ]:
results

# Now, it's your turn

We use CTCF binding site as example but you can also use it for many other tasks such as promoter prediction, structural variants, integration sites of transposable elements, other binding sites, and many more.